# Capstone Project - Agentic AI: Bank Customer Churn Analytics
### Multi-Agent Intelligent Insights Engine with Gemini-Powered Q&A on Bank Cystomer Churn Kaggle Dataset
### Version: 1.0
### Created by: Omar Chehab
### Date: 30-11-2025

## Project Overview
An advanced **agentic AI system** that transforms raw bank customer churn data into actionable executive insights. This project demonstrates a multi-agent architecture where specialized AI agents work autonomously to deliver intelligent analysis:

### 🤖 Agent Architecture Overview:

1. **Analytics Agent**: The primary intelligence engine that autonomously:
   - Performs multi-dimensional analysis on customer churn data (geographic, demographic, financial)
   - Generates executive summaries and risk assessments
   - Prepares comprehensive context for decision-making

2. **Web Search Agent**: An optional secondary agent that autonomously:
   - Searches for industry benchmarks and market trends when relevant
   - Enriches internal insights with external market intelligence
   - Filters queries to ensure relevance to churn analysis

3. **Gemini LLM Agent**: The orchestrator that:
   - Synthesizes insights from both Analytics and Web Search agents
   - Generates natural language responses tailored to executive audience
   - Adapts communication style based on context and data insights

**Key Agentic Capabilities**:
- ✅ Autonomous decision-making within defined domains
- ✅ Real-time data analysis and context generation
- ✅ Optional web search integration for market insights
- ✅ Conversational Q&A with multi-turn dialogue support
- ✅ Executive-level reporting with actionable recommendations

## 1. Importing Required Modules and Libraries
Import necessary libraries for data manipulation, API configuration, and Google ADK components for agent-based processing.

In [29]:
from typing import Any, Dict
import os
from kaggle_secrets import UserSecretsClient
import pandas as pd

from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.runners import InMemoryRunner
from google import generativeai as genai


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## 2. API Configuration & Agent Setup
Configure Google Generative AI credentials from Kaggle Secrets and establish model parameters for the analytics agent.

In [30]:
# Load API Key from Kaggle Secrets
import os
import google.generativeai as genai

try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    
    # Configure genai with the API key
    genai.configure(api_key=GOOGLE_API_KEY)
    
    print("✅ Gemini API key setup complete.")
except ImportError:
    print("⚠️ Kaggle Secrets not available. Ensure you're in a Kaggle Notebook.")
except KeyError:
    print("🔑 Authentication Error: Add 'GOOGLE_API_KEY' to Kaggle secrets.")

# Agent Configuration
CONFIG = {
    "project": "",
    "model": "models/gemini-2.5-flash",
    "max_tokens": 2000,
    "temperature": 0.3,
    "version": "1.0"
}

print(f"\n{'='*60}")
print(f"{'AGENT CONFIGURATION':^60}")
print(f"{'='*60}")
for k, v in CONFIG.items():
    print(f"{k:.<25} {v}")
print(f"{'='*60}")

✅ Gemini API key setup complete.

                    AGENT CONFIGURATION                     
project.................. 
model.................... models/gemini-2.5-flash
max_tokens............... 2000
temperature.............. 0.3
version.................. 1.0


In [31]:
retry_config = types.HttpRetryOptions(
    attempts=5, # Maximum retry attempts
    exp_base=2, # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504] # HTTP errors retry
)

print("✅ Retry configuration defined.")

✅ Retry configuration defined.


In [32]:
# Lists all LLM models available in your Google Generative AI account
models = genai.list_models()

# Prints each model name
for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
mod

## 3. Data Loading & Initial Exploration
Load the Bank Customer Churn Prediction dataset from Kaggle and perform initial data quality checks.

In [33]:
# Loading DataFrame
# Dataset URL - https://www.kaggle.com/datasets/saurabhbadole/bank-customer-churn-prediction-dataset
df = pd.read_csv("/kaggle/input/bank-customer-churn-prediction-dataset/Churn_Modelling.csv")

# Quick check
print(f"✅ Data loaded: {len(df)} rows")
print(f"📊 Columns: {df.columns.tolist()}")

✅ Data loaded: 10000 rows
📊 Columns: ['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']


In [34]:
# Checking sample output
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 4. Core Analytics Agent Implementation
Define the `AnalyticsAgent` class that performs multi-dimensional analysis on customer churn data:
- **Executive Summary**: Key metrics and overall statistics
- **Geographic Analysis**: Churn patterns by country/region
- **Demographic Analysis**: Age group and gender-based churn insights
- **Product Engagement**: Product count and member activity impact
- **Risk Segmentation**: Identification of high-risk customer groups
- **Financial Profiling**: Comparison of churned vs retained customers
- **Customer Lifetime Value Impact**: Revenue loss analysis
- **Gemini-Powered Q&A**: Natural language interface with optional web search

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Any
import google.generativeai as gen
from kaggle_secrets import UserSecretsClient

class AnalyticsAgent:
    """
    Analytics Agent for generating executive insights on Bank Customer Churn data.
    Powered by Google Gemini for intelligent Q&A with web search capability.
    """
    
    def __init__(self, dataframe: pd.DataFrame):
        """
        Initialize the Analytics Agent with the dataset.
        
        Args:
            dataframe: pandas DataFrame containing bank customer churn data
        """
        self.df = dataframe. copy()
        self.insights = {}
        self.chat_history = []
        
    def generate_executive_summary(self) -> Dict[str, Any]:
        """Generate a comprehensive executive summary with key metrics."""
        summary = {
            'total_customers': int(len(self.df)),
            'churn_rate': float(self._calculate_churn_rate()),
            'avg_customer_age': float(self. df['Age'].mean()),
            'avg_account_balance': float(self.df['Balance'].mean()),
            'avg_estimated_salary': float(self.df['EstimatedSalary'].mean()),
            'active_member_rate': float((self.df['IsActiveMember']. sum() / len(self.df)) * 100),
            'credit_card_holder_rate': float((self.df['HasCrCard'].sum() / len(self.df)) * 100),
        }
        self.insights['executive_summary'] = summary
        return summary
    
    def _calculate_churn_rate(self) -> float:
        """Calculate overall churn rate as percentage."""
        return (self.df['Exited'].sum() / len(self.df)) * 100
    
    def analyze_churn_by_geography(self) -> pd.DataFrame:
        """Analyze churn rates across different geographies."""
        geo_analysis = self.df.groupby('Geography', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'EstimatedSalary': 'mean'
        }).round(2)
        geo_analysis.columns = ['Churned_Customers', 'Total_Customers', 'Churn_Rate', 
                                'Avg_Balance', 'Avg_Salary']
        geo_analysis['Churn_Rate'] = geo_analysis['Churn_Rate'] * 100
        self.insights['geography_analysis'] = geo_analysis
        return geo_analysis
    
    def analyze_churn_by_demographics(self) -> Dict[str, pd.DataFrame]:
        """Analyze churn patterns by demographic factors (age, gender)."""
        self.df['AgeGroup'] = pd.cut(self.df['Age'], 
                                      bins=[0, 30, 40, 50, 60, 100],
                                      labels=['<30', '30-40', '40-50', '50-60', '60+'])
        
        age_analysis = self.df.groupby('AgeGroup', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean'
        }).round(2)
        age_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance']
        age_analysis['Churn_Rate'] = age_analysis['Churn_Rate'] * 100
        
        gender_analysis = self.df.groupby('Gender', observed=True). agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'CreditScore': 'mean'
        }).round(2)
        gender_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance', 'Avg_CreditScore']
        gender_analysis['Churn_Rate'] = gender_analysis['Churn_Rate'] * 100
        
        demographics = {
            'age_analysis': age_analysis,
            'gender_analysis': gender_analysis
        }
        self.insights['demographics'] = demographics
        return demographics
    
    def analyze_product_engagement(self) -> pd. DataFrame:
        """Analyze churn based on number of products and engagement metrics."""
        product_analysis = self.df.groupby('NumOfProducts', observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'Tenure': 'mean',
            'IsActiveMember': 'mean'
        }).round(2)
        product_analysis.columns = ['Churned', 'Total', 'Churn_Rate', 
                                    'Avg_Balance', 'Avg_Tenure', 'Active_Rate']
        product_analysis['Churn_Rate'] = product_analysis['Churn_Rate'] * 100
        product_analysis['Active_Rate'] = product_analysis['Active_Rate'] * 100
        self.insights['product_engagement'] = product_analysis
        return product_analysis
    
    def identify_high_risk_segments(self) -> pd.DataFrame:
        """Identify customer segments with highest churn risk."""
        segments = self.df.groupby(['Geography', 'Gender', 'IsActiveMember'], observed=True).agg({
            'Exited': ['sum', 'count', 'mean'],
            'Balance': 'mean',
            'Age': 'mean'
        }).round(2)
        segments.columns = ['Churned', 'Total', 'Churn_Rate', 'Avg_Balance', 'Avg_Age']
        segments['Churn_Rate'] = segments['Churn_Rate'] * 100
        high_risk = segments[segments['Total'] >= 50]. sort_values('Churn_Rate', ascending=False)
        self.insights['high_risk_segments'] = high_risk. head(10)
        return high_risk. head(10)
    
    def analyze_financial_profile(self) -> Dict[str, Any]:
        """Analyze financial characteristics of churned vs retained customers."""
        churned = self.df[self.df['Exited'] == 1]
        retained = self.df[self.df['Exited'] == 0]
        
        financial_profile = {
            'churned_customers': {
                'avg_balance': float(churned['Balance'].mean()),
                'median_balance': float(churned['Balance'].median()),
                'avg_credit_score': float(churned['CreditScore'].mean()),
                'avg_salary': float(churned['EstimatedSalary'].mean()),
                'zero_balance_pct': float((churned['Balance'] == 0).sum() / len(churned) * 100)
            },
            'retained_customers': {
                'avg_balance': float(retained['Balance']. mean()),
                'median_balance': float(retained['Balance'].median()),
                'avg_credit_score': float(retained['CreditScore'].mean()),
                'avg_salary': float(retained['EstimatedSalary'].mean()),
                'zero_balance_pct': float((retained['Balance'] == 0).sum() / len(retained) * 100)
            }
        }
        self.insights['financial_profile'] = financial_profile
        return financial_profile
    
    def calculate_customer_lifetime_value_impact(self) -> Dict[str, float]:
        """Calculate the financial impact of customer churn."""
        churned = self.df[self.df['Exited'] == 1]
        estimated_revenue_per_customer = 0.01
        
        impact = {
            'total_churned_customers': int(len(churned)),
            'total_balance_lost': float(churned['Balance'].sum()),
            'avg_balance_per_churned_customer': float(churned['Balance'].mean()),
            'estimated_annual_revenue_loss': float(churned['Balance'].sum() * estimated_revenue_per_customer),
            'avg_tenure_of_churned': float(churned['Tenure'].mean()),
        }
        self.insights['clv_impact'] = impact
        return impact
    
    def get_all_insights(self) -> Dict[str, Any]:
        """Run all analyses and return comprehensive insights dictionary."""
        self.generate_executive_summary()
        self.analyze_churn_by_geography()
        self.analyze_churn_by_demographics()
        self.analyze_product_engagement()
        self.identify_high_risk_segments()
        self.analyze_financial_profile()
        self.calculate_customer_lifetime_value_impact()
        return self.insights
    
    # ============================================================================
    # GEMINI-POWERED Q&A FUNCTIONALITY WITH WEB SEARCH
    # ============================================================================
    
    def _prepare_context(self) -> str:
        """
        Prepare a comprehensive context string with all insights for Gemini.
        """
        if not self.insights:
            self.get_all_insights()
        
        context = "=== BANK CUSTOMER CHURN ANALYSIS DATA ===\n\n"
        
        # Executive Summary
        context += "EXECUTIVE SUMMARY:\n"
        for key, value in self.insights['executive_summary'].items():
            context += f"- {key. replace('_', ' ').title()}: {value:,.2f}\n"
        
        # Geography Analysis
        context += "\nCHURN BY GEOGRAPHY:\n"
        geo_df = self.insights['geography_analysis']
        for geo, row in geo_df.iterrows():
            context += f"- {geo}: {row['Churn_Rate']:.2f}% churn rate, "
            context += f"{int(row['Churned_Customers'])} of {int(row['Total_Customers'])} customers, "
            context += f"Avg Balance: ${row['Avg_Balance']:,.2f}\n"
        
        # Demographics - Age
        context += "\nCHURN BY AGE GROUP:\n"
        age_df = self.insights['demographics']['age_analysis']
        for age_group, row in age_df.iterrows():
            context += f"- {age_group}: {row['Churn_Rate']:.2f}% churn rate, "
            context += f"{int(row['Churned'])} of {int(row['Total'])} customers\n"
        
        # Demographics - Gender
        context += "\nCHURN BY GENDER:\n"
        gender_df = self. insights['demographics']['gender_analysis']
        for gender, row in gender_df.iterrows():
            context += f"- {gender}: {row['Churn_Rate']:.2f}% churn rate, "
            context += f"{int(row['Churned'])} of {int(row['Total'])} customers\n"
        
        # Product Engagement
        context += "\nCHURN BY NUMBER OF PRODUCTS:\n"
        product_df = self.insights['product_engagement']
        for num_products, row in product_df.iterrows():
            context += f"- {int(num_products)} products: {row['Churn_Rate']:.2f}% churn rate, "
            context += f"{int(row['Churned'])} of {int(row['Total'])} customers, "
            context += f"Avg Tenure: {row['Avg_Tenure']:.1f} years, "
            context += f"Active Rate: {row['Active_Rate']:.1f}%\n"
        
        # High Risk Segments
        context += "\nTOP 5 HIGH-RISK SEGMENTS:\n"
        high_risk = self.insights['high_risk_segments']. head(5)
        for idx, (segment, row) in enumerate(high_risk.iterrows(), 1):
            geo, gender, is_active = segment
            active_status = "Active" if is_active == 1 else "Inactive"
            context += f"{idx}. {geo} - {gender} - {active_status}: "
            context += f"{row['Churn_Rate']:.2f}% churn rate, {int(row['Total'])} customers\n"
        
        # Financial Profile
        context += "\nFINANCIAL PROFILE COMPARISON:\n"
        fp = self.insights['financial_profile']
        context += "Churned Customers:\n"
        context += f"  - Avg Balance: ${fp['churned_customers']['avg_balance']:,.2f}\n"
        context += f"  - Avg Credit Score: {fp['churned_customers']['avg_credit_score']:.0f}\n"
        context += f"  - Avg Salary: ${fp['churned_customers']['avg_salary']:,.2f}\n"
        context += "Retained Customers:\n"
        context += f"  - Avg Balance: ${fp['retained_customers']['avg_balance']:,.2f}\n"
        context += f"  - Avg Credit Score: {fp['retained_customers']['avg_credit_score']:.0f}\n"
        context += f"  - Avg Salary: ${fp['retained_customers']['avg_salary']:,.2f}\n"
        
        # Financial Impact
        context += "\nFINANCIAL IMPACT:\n"
        impact = self.insights['clv_impact']
        context += f"- Total Churned Customers: {impact['total_churned_customers']:,}\n"
        context += f"- Total Balance Lost: ${impact['total_balance_lost']:,.2f}\n"
        context += f"- Estimated Annual Revenue Loss: ${impact['estimated_annual_revenue_loss']:,.2f}\n"
        context += f"- Avg Tenure of Churned: {impact['avg_tenure_of_churned']:.1f} years\n"
        
        return context
    
    def _create_system_prompt(self) -> str:
        """
        Create the system prompt that defines the agent's role and behavior.
        """
        system_prompt = """You are an expert Analytics Agent specializing in providing insights to executive based on the customer churn analysis for a bank. 

                        Your role is to answer executive questions about customer churn data with:
                        - Clear, concise, and actionable insights
                        - Data-driven responses based on the provided analysis
                        - Executive-friendly language (avoid jargon)
                        - Specific numbers and percentages from the data
                        - Strategic recommendations when appropriate
                        - Professional formatting with emojis for visual clarity (📊 🌍 👥 💰 ⚠️ 🎯)
                        - Make sure you are not using any jargon or complicated words, keep the wording very simple and clear
                        - If you are asked a question beyond the scope of the dataset make sure to mention that you can only answer questions based on the customer churn data
                        
                        When answering:
                        1. Always reference specific data points from the analysis
                        2. Highlight key insights and patterns
                        3. Provide context and comparisons
                        4. End with actionable recommendations when relevant
                        5. Be direct and avoid unnecessary preamble
                        6. Use bullet points and clear structure
                        7. If asked a question beyond the scope of the dataset, mention that this question is out of scope
                        
                        The data context below contains all the churn analysis results you should reference. 
                        """
        return system_prompt
    
    def model_config(self, system_prompt: str, user_prompt: str) -> str:
        """
        Configure and call Gemini model with the given prompts.
        
        Args:
            system_prompt: System instructions for the model
            user_prompt: User's question
            
        Returns:
            Model's response text
        """
        try:
            api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        except Exception:
            return "[Simulated LLM: GOOGLE_API_KEY secret not accessible in this environment.]"
        
        try:    
            gen. configure(api_key=api_key)
            model = gen.GenerativeModel("gemini-2.0-flash-exp")
            prompt = system_prompt + "\n\n" + user_prompt
            response = model.generate_content(prompt)
            return response.text
                
        except Exception as e:
            return f"[Simulated LLM: Gemini unreachable → {e}]"
    
    def _is_churn_related_query(self, question: str) -> bool:
        """
        Determine if a query is relevant to customer churn analysis.
        Only perform web searches for churn-related topics.
        
        Args:
            question: User's question
            
        Returns:
            True if query is relevant to churn analysis, False otherwise
        """
        churn_keywords = [
            'churn', 'retention', 'customer attrition', 'bank', 'finance',
            'customer behavior', 'risk', 'segment', 'loyalty', 'satisfaction',
            'industry', 'trend', 'benchmark', 'best practice', 'strategy',
            'retention rate', 'attrition rate', 'customer lifetime value',
            'clv', 'revenue', 'profitability', 'financial', 'demographic'
        ]
        
        question_lower = question.lower()
        return any(keyword in question_lower for keyword in churn_keywords)
    
    def _search_external_info(self, question: str) -> str:
        """
        Search for external information relevant to the churn analysis query.
        Uses Google Search to find industry benchmarks, trends, and best practices.
        
        Args:
            question: User's question for web search
            
        Returns:
            String containing search results or empty string if search fails
        """
        # Only search if query is churn-related
        if not self._is_churn_related_query(question):
            return ""
        
        try:
            # Extract churn-relevant keywords from the question
            search_query = f"customer churn banking {question}"
            
            # Attempt to use google_search tool
            from google.adk.tools import google_search
            
            search_results = google_search(search_query)
            
            if search_results:
                # Format search results into context
                formatted_results = "📱 EXTERNAL INSIGHTS FROM WEB SEARCH:\n"
                for result in search_results[:3]:  # Limit to top 3 results
                    if hasattr(result, 'title') and hasattr(result, 'snippet'):
                        formatted_results += f"- {result.title}: {result.snippet}\n"
                return formatted_results
            else:
                return ""
                
        except Exception as e:
            # Silently fail - proceed without external search
            return ""
    
    def ask_with_search(self, question: str) -> str:
        """
        Ask a question with external web search capability.
        Searches Google for relevant churn industry trends, benchmarks, and best practices.
        Only searches for queries related to customer churn analysis.
        
        Args:
            question: Natural language question from the executive
            
        Returns:
            String answer combining dataset insights with external information
        """
        # Ensure insights are generated
        if not self.insights:
            print("📊 Analyzing data... Please wait...")
            self.get_all_insights()
            print("✅ Analysis complete!\n")
        
        # Store question in chat history
        self.chat_history.append({'role': 'user', 'content': question})
        
        # Try to fetch external information if relevant
        print("🔍 Searching for external insights...")
        external_info = self._search_external_info(question)
        
        # Prepare context and prompts
        context = self._prepare_context()
        system_prompt = self._create_system_prompt()
        
        # Enhance prompt with external search results if available
        if external_info:
            user_prompt = f"""DATA CONTEXT:\n{context}\n\n{external_info}\n\nEXECUTIVE QUESTION:\n{question}\n\nCombine internal data insights with the external research to provide a comprehensive answer:"""
        else:
            user_prompt = f"DATA CONTEXT:\n{context}\n\nEXECUTIVE QUESTION:\n{question}\n\nProvide a clear, data-driven answer:"
        
        # Get response from Gemini
        answer = self.model_config(system_prompt, user_prompt)
        
        # Store answer in chat history
        self.chat_history.append({'role': 'agent', 'content': answer})
        
        return answer
    
    def ask(self, question: str) -> str:
        """
        Main Q&A interface.  Ask the agent any question about the churn data.
        Powered by Google Gemini LLM.
        
        Args:
            question: Natural language question from the executive
            
        Returns:
            String answer to the question
        """
        # Ensure insights are generated
        if not self.insights:
            print("📊 Analyzing data... Please wait...")
            self.get_all_insights()
            print("✅ Analysis complete!\n")
        
        # Store question in chat history
        self.chat_history.append({'role': 'user', 'content': question})
        
        # Prepare context and prompts
        context = self._prepare_context()
        system_prompt = self._create_system_prompt()
        user_prompt = f"DATA CONTEXT:\n{context}\n\nEXECUTIVE QUESTION:\n{question}\n\nProvide a clear, data-driven answer:"
        
        # Get response from Gemini
        answer = self.model_config(system_prompt, user_prompt)
        
        # Store answer in chat history
        self.chat_history.append({'role': 'agent', 'content': answer})
        
        return answer
    
    def start_chat(self):
        """
        Start an interactive chat session (for Jupyter notebooks or console).
        Type 'quit', 'exit', or 'bye' to end the session.
        """
        print("=" * 80)
        print("🤖 ANALYTICS AGENT - EXECUTIVE Q&A SESSION (Powered by Gemini)")
        print("=" * 80)
        print("\nHello! I'm your AI-powered Analytics Agent. I can answer questions about")
        print("customer churn using advanced language understanding.")
        print("\nType 'quit', 'exit', or 'bye' to end the session.\n")
        print("-" * 80)
        
        # Ensure insights are loaded
        if not self.insights:
            print("📊 Loading and analyzing data... Please wait...")
            self.get_all_insights()
            print("✅ Ready to answer your questions!\n")
        
        while True:
            try:
                question = input("\n💼 Executive: ").strip()
                
                if question.lower() in ['quit', 'exit', 'bye', 'q']:
                    print("\n👋 Thank you for using Analytics Agent.  Goodbye!")
                    break
                
                if not question:
                    continue
                
                print("\n🤖 Agent: [Thinking... ]\n")
                answer = self. ask(question)
                print(answer)
                print("\n" + "-" * 80)
                
            except KeyboardInterrupt:
                print("\n\n👋 Session ended.  Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ Error: {str(e)}")
                print("Please try rephrasing your question.\n")
    
    def get_chat_history(self) -> List[Dict[str, str]]:
        """Return the chat history."""
        return self.chat_history
    
    def clear_chat_history(self):
        """Clear the chat history."""
        self.chat_history = []
        print("✅ Chat history cleared.")
    
    def print_executive_report(self):
        """Print a formatted executive report to console."""
        insights = self.get_all_insights()
        
        print("=" * 80)
        print("EXECUTIVE INSIGHTS REPORT - BANK CUSTOMER CHURN ANALYSIS")
        print("=" * 80)
        
        print("\n📈 EXECUTIVE SUMMARY:")
        print("-" * 80)
        for key, value in insights['executive_summary'].items():
            key_formatted = key.replace('_', ' ').title()
            if isinstance(value, int):
                print(f"  {key_formatted}: {value:,}")
            else:
                print(f"  {key_formatted}: {value:,.2f}")
        
        print("\n\n🌍 CHURN BY GEOGRAPHY:")
        print("-" * 80)
        print(insights['geography_analysis'])
        
        print("\n\n📦 PRODUCT ENGAGEMENT:")
        print("-" * 80)
        print(insights['product_engagement'])
        
        print("\n\n⚠️  TOP 5 HIGH-RISK SEGMENTS:")
        print("-" * 80)
        print(insights['high_risk_segments'].head())
        
        print("\n" + "=" * 80)

print("✅ AnalyticsAgent class loaded successfully!")

✅ AnalyticsAgent class loaded successfully!


In [48]:
# Create the Analytics Agent
agent = AnalyticsAgent(df)

print("✅ Agent initialized and ready!")

✅ Agent initialized and ready!


## 5. Agent Initialization & General Q&A Demo
Initialize the analytics agent with the churn dataset and demonstrate its Q&A capabilities with foundational questions.

In [49]:
agent.ask("What can you help with? What type of dataset can you support me with?")

📊 Analyzing data... Please wait...
✅ Analysis complete!



'I can help you understand why customers are leaving the bank and what actions can be taken to keep them. \n\nI can support you with this customer churn data, which includes:\n\n*   **Customer Demographics:** Age, gender, and location (France, Germany, Spain). 🌍\n*   **Account Information:** Account balance, number of products, credit card ownership, and active member status. 💰\n*   **Churn Status:** Whether a customer has left the bank or not. ⚠️\n*   **Financial Details:** Credit score and estimated salary. 📊\n*   **Tenure:** How long a customer has been with the bank. 👥\n\nUsing this data, I can identify trends, high-risk groups, and potential areas for improvement to reduce customer churn. 🎯\n'

## 6. Interactive Q&A Demonstrations
Ask the agent specific questions about churn patterns, demographics, and business insights. These demonstrations showcase different analysis dimensions.

In [50]:
agent.ask("What's our overall churn rate?")

'The overall churn rate for our bank is 20.37%. 📊 That means out of 10,000 customers, 2,037 have left. We need to look into why so many customers are leaving. ⚠️\n'

In [51]:
agent.ask("What's our biggest churn problem?")

"Okay, here's a breakdown of our biggest churn problem based on the data:\n\nOur biggest churn problem is **customers in Germany**. 🌍 Here’s why:\n\n*   **Highest Churn Rate:** Germany has a churn rate of 32.00%. This is much higher compared to France (16.00%) and Spain (17.00%).\n*   **Significant Customer Loss:** We lost 814 customers in Germany.\n*   **High-Risk Segments:** Three out of the top five high-risk customer segments are in Germany:\n    *   German Female Inactive: 45.00% churn\n    *   German Male Inactive: 37.00% churn\n    *   German Female Active: 30.00% churn\n*   **Balance Impact:** Even though the balance lost per customer isn't specified for Germany alone, the overall balance lost from churn is significant at $185,588,094.63, and Germany contributes substantially to this.\n\n**Recommendation:** 🎯\nWe need to investigate why German customers are leaving at such a high rate. We should look into regional differences in customer service, product offerings, and marketin

In [52]:
agent.ask("Which factors contribute most to churn?")

"Okay, here's a breakdown of the key factors driving customer churn at our bank, based on the data you provided:\n\n*   **Geography:** 🌍\n    *   Germany has a high churn rate of 32%, significantly higher than France (16%) and Spain (17%).\n*   **Age:** 🎂\n    *   Customers in the 40-50 age group have a 34% churn rate.\n    *   The 50-60 age group shows an even higher churn rate of 56%.\n*   **Gender:** 🧑\u200d🤝\u200d🧑\n    *   Female customers churn at a higher rate (25%) compared to male customers (16%).\n*   **Number of Products:** 🛒\n    *   Customers with only 1 product have a churn rate of 28%.\n    *   Customers with 3 or 4 products have extremely high churn rates of 83% and 100% respectively, though these groups represent smaller segments of our customer base.\n*   **Inactive Status:** 😴\n    *   Inactive members in Germany and other regions show very high churn rates, especially among female customers.\n\n**Key Insights:**\n\n*   German customers, particularly inactive females

In [53]:
# # Demographics
agent.ask("How does age affect churn?")

'Age has a big impact on customer churn. Here’s a breakdown 📊:\n\n*   **Youngest Customers (\\<30):** Low churn at 8.00%, or 148 out of 1968 customers.\n*   **30-40 Age Group:** Slightly higher churn at 12.00%, with 538 out of 4451 customers leaving.\n*   **40-50 Age Group:** Churn more than doubles to 34.00%, with 788 out of 2320 customers churning.\n*   **50-60 Age Group:** Highest churn rate at 56.00%, where 448 out of 797 customers churn.\n*   **Oldest Customers (60+):** Churn decreases to 25.00%, with 115 out of 464 customers.\n\n**Key Insight:** 🌍\n\n*   Churn is highest for customers in the 40-60 age range, especially those in their 50s.\n\n**Recommendation:** 🎯\n\n*   Focus retention efforts on customers aged 40-60. Understand their specific needs and concerns to reduce churn. Look into what products or services might appeal to this age group to keep them as customers.'

In [54]:
agent.ask("Do men or women churn more?")

"📊 Here's the breakdown of churn by gender:\n\n*   **Female Churn:** 25.00% of female customers churn (1139 out of 4543).\n*   **Male Churn:** 16.00% of male customers churn (898 out of 5457).\n\n🌍 **Insight:** Women churn at a higher rate than men.\n\n🎯 **Recommendation:** Investigate why women are more likely to leave. Look into service satisfaction, product alignment, or other factors that may be driving this difference.\n"

In [55]:
agent.ask("What age group should we focus on?")

"Okay, here's the age group we should focus on to reduce churn:\n\n*   **Key Insight:** The highest churn rates are among customers aged 40-60. ⚠️\n\n*   **Data Breakdown:**\n    *   40-50 age group: 34% churn rate (788 of 2,320 customers).\n    *   50-60 age group: 56% churn rate (448 of 797 customers).\n    *   Comparatively, younger customers (<30 and 30-40) have much lower churn (8% and 12% respectively). 📊\n\n*   **Why this matters:** These two age groups (40-60) represent a large portion of our churned customers. Addressing their needs can significantly impact our overall churn rate. 👥\n\n*   **Recommendation:** Focus retention efforts on customers in the 40-60 age brackets. Investigate the reasons behind their high churn rates (e.g., changing life priorities, retirement planning, etc.) and tailor services/products to better meet their needs. 🎯\n"

## 7. Interactive Visual Chatbot Interface
Create a user-friendly web-based chatbot UI using Jupyter Widgets with the following features:
- **Real-time chat interface** with message history
- **Quick question buttons** for common inquiries
- **Web search toggle** for enriched insights with external data
- **Responsive design** with color-coded messages
- **Gemini-powered responses** formatted for executive consumption

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

class VisualChatbot:
    """Visual chatbot using Jupyter widgets with web search capability"""
    
    def __init__(self, agent):
        self.agent = agent
        self.chat_history = []
        self.use_web_search = False  # Toggle for web search
        self.setup_gemini()
        self.create_ui()
    
    def setup_gemini(self):
        """Setup Gemini API"""
        try:
            api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
            genai.configure(api_key=api_key)
            
            model = genai.GenerativeModel(
                model_name="gemini-2.0-flash-exp",
                generation_config={
                    "temperature": 0.7,
                    "top_p": 0.95,
                    "max_output_tokens": 8192,
                }
            )
            
            context = self.agent._prepare_context()
            system_instruction = f"""You are an expert Analytics Agent specializing in bank customer churn analysis. 

{context}

Provide clear, data-driven insights with specific numbers and actionable recommendations."""
            
            self.chat_session = model.start_chat(history=[])
            self.chat_session. send_message(system_instruction)
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def create_ui(self):
        """Create widget-based UI with better contrast and web search toggle"""
        # Chat output area - uses flex to fill available space
        self.chat_output = widgets.Output(
            layout=widgets.Layout(
                min_height='500px',
                max_height='800px',
                border='2px solid #667eea',
                padding='15px',
                overflow_y='auto',
                background_color='#ffffff',
                flex='1 1 auto'
            )
        )
        
        # Input box
        self.input_box = widgets.Text(
            placeholder='Type your question here...',
            layout=widgets.Layout(width='70%'),
            style={'description_width': 'initial'}
        )
        
        # Send button
        self.send_button = widgets.Button(
            description='Send 📤',
            button_style='primary',
            layout=widgets.Layout(width='15%')
        )
        
        # Web Search Toggle Button with ON/OFF status
        self.search_toggle = widgets.ToggleButton(
            value=False,
            description='🔍 Search Web: OFF',
            button_style='danger',  # Red for OFF
            layout=widgets.Layout(width='18%'),
            tooltip='Enable external web search for churn-related queries'
        )
        
        # Clear button
        self.clear_button = widgets.Button(
            description='Clear Chat 🗑️',
            button_style='warning',
            layout=widgets.Layout(width='100%', margin='10px 0')
        )
        
        # Quick question buttons with full text visible
        button_style = widgets.Layout(width='auto', min_width='200px', margin='5px')
        self.quick_buttons = [
            widgets.Button(description="📊 What's our churn rate?", button_style='info', layout=button_style),
            widgets.Button(description="⚠️ High-risk segments?", button_style='info', layout=button_style),
            widgets.Button(description="💡 Recommendations?", button_style='info', layout=button_style),
            widgets.Button(description="💰 Financial impact?", button_style='info', layout=button_style),
            widgets.Button(description="🌍 Geography analysis?", button_style='info', layout=button_style),
            widgets.Button(description="📋 Executive summary?", button_style='info', layout=button_style),
        ]
        
        # Event handlers
        self.send_button.on_click(self.on_send)
        self.input_box.on_submit(self.on_send)
        self.clear_button.on_click(self.on_clear)
        self.search_toggle.observe(self.on_search_toggle, names='value')
        
        # Quick button handlers
        self.quick_buttons[0].on_click(lambda b: self.send_quick_question("What's our overall churn rate?"))
        self.quick_buttons[1].on_click(lambda b: self.send_quick_question("Who are our high-risk customer segments?"))
        self.quick_buttons[2]. on_click(lambda b: self.send_quick_question("What are your top recommendations to reduce churn?"))
        self.quick_buttons[3].on_click(lambda b: self.send_quick_question("What's the financial impact of churn?"))
        self.quick_buttons[4].on_click(lambda b: self. send_quick_question("Which geography has the highest churn?"))
        self.quick_buttons[5].on_click(lambda b: self.send_quick_question("Give me an executive summary"))
        
        # Display welcome message
        with self.chat_output:
            display(HTML("""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
                <h2 style="margin: 0; color: white;">🤖 Bank Customer Churn Analytics Agent</h2>
                <p style="margin: 10px 0; color: white; font-size: 16px;">Powered by Google Gemini AI with Optional Web Search</p>
                <p style="font-size: 14px; opacity: 0.95; color: white;">Ask me anything about customer churn patterns and recommendations! </p>
            </div>
            """))
    
    def on_search_toggle(self, change):
        """Handle web search toggle - updates button with ON/OFF status"""
        self.use_web_search = change['new']
        
        if self.use_web_search:
            # Green for ON
            self.search_toggle.description = '🔍 Search Web: ON'
            self.search_toggle.button_style = 'success'
            status_text = '🟢 Search Web: ON'
        else:
            # Red for OFF
            self.search_toggle.description = '🔍 Search Web: OFF'
            self.search_toggle.button_style = 'danger'
            status_text = '🔴 Search Web: OFF'
        
        print(f"Web Search {status_text}")
    
    def display(self):
        """Display the chat interface"""
        # Header with better styling
        header = widgets.HTML("""
        <div style="margin-bottom: 15px; padding: 10px; background: #f8f9fa; border-radius: 8px; border-left: 4px solid #667eea;">
            <h3 style="margin: 0; color: #333;">💡 Quick Questions (Click to Ask):</h3>
        </div>
        """)
        
        # Layout quick buttons in rows with better spacing
        quick_buttons_row1 = widgets.HBox([self.quick_buttons[0], self.quick_buttons[1], self.quick_buttons[2]], 
                                          layout=widgets.Layout(justify_content='flex-start', margin='5px 0'))
        quick_buttons_row2 = widgets.HBox([self.quick_buttons[3], self.quick_buttons[4], self.quick_buttons[5]], 
                                          layout=widgets.Layout(justify_content='flex-start', margin='5px 0'))
        
        quick_buttons_box = widgets.VBox([quick_buttons_row1, quick_buttons_row2])
        
        # Chat label
        chat_label = widgets.HTML("""
        <div style='margin: 20px 0 10px 0; padding: 10px; background: #f8f9fa; border-radius: 8px; border-left: 4px solid #667eea;'>
            <strong style="color: #333;">💬 Chat:</strong>
        </div>
        """)
        
        # Chat area wrapper - allows chat to expand and scroll
        chat_area = widgets.VBox(
            [chat_label, self.chat_output],
            layout=widgets.Layout(flex='1 1 auto', min_height='350px')
        )
        
        # Input section - fixed at bottom with search toggle
        input_section = widgets.VBox(
            [
                widgets.HBox(
                    [self.input_box, self.send_button, self.search_toggle],
                    layout=widgets.Layout(width='100%', margin='10px 0 0 0')
                ),
                self.clear_button
            ],
            layout=widgets.Layout(flex='0 0 auto')
        )
        
        # Complete UI with flex layout
        ui = widgets.VBox(
            [
                header,
                quick_buttons_box,
                chat_area,
                input_section
            ],
            layout=widgets.Layout(
                display='flex',
                flex_flow='column',
                height='900px'
            )
        )
        
        display(ui)
    
    def on_send(self, b):
        """Handle send button click"""
        message = self. input_box.value. strip()
        if message:
            self.send_message(message)
            self.input_box.value = ''
    
    def send_quick_question(self, question):
        """Send a quick question"""
        self.send_message(question)
    
    def send_message(self, message):
        """Send message and display response with web search capability"""
        # Display user message with dark text
        with self.chat_output:
            display(HTML(f"""
            <div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #2196f3; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                <strong style="color: #1565c0; font-size: 14px;">💼 You:</strong>
                <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.6;">{message}</p>
            </div>
            """))
        
        # Show thinking indicator
        with self.chat_output:
            search_status = "🔍 with Web Search" if self.use_web_search else ""
            display(HTML(f"""
            <div style="background: #fff9c4; padding: 10px; border-radius: 8px; margin: 10px 0; text-align: center;">
                <span style="color: #f57c00;">🤖 Agent is thinking {search_status}...</span>
            </div>
            """))
        
        # Get response
        try:
            # Use web search if enabled
            if self.use_web_search:
                response_text = self.agent.ask_with_search(message)
            else:
                response_text = self.agent.ask(message)
            
            # Clear thinking indicator and display agent response with dark text
            self.chat_output.clear_output(wait=True)
            
            # Re-display all chat history
            for item in self.chat_history:
                with self.chat_output:
                    display(HTML(f"""
                    <div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #2196f3; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                        <strong style="color: #1565c0; font-size: 14px;">💼 You:</strong>
                        <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.6;">{item['user']}</p>
                    </div>
                    """))
                    
                    # Convert markdown-style formatting to HTML
                    formatted_response = item['agent']. replace('\n', '<br>')
                    formatted_response = formatted_response.replace('**', '<strong>').replace('**', '</strong>')
                    
                    display(HTML(f"""
                    <div style="background: #f5f5f5; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #667eea; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                        <strong style="color: #5e35b1; font-size: 14px;">🤖 Agent:</strong>
                        <div style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1. 8;">{formatted_response}</div>
                    </div>
                    """))
            
            # Display current message and response
            with self.chat_output:
                display(HTML(f"""
                <div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #2196f3; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <strong style="color: #1565c0; font-size: 14px;">💼 You:</strong>
                    <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.6;">{message}</p>
                </div>
                """))
                
                # Convert markdown-style formatting to HTML
                formatted_response = response_text.replace('\n', '<br>')
                formatted_response = formatted_response.replace('**', '<strong>').replace('**', '</strong>')
                
                display(HTML(f"""
                <div style="background: #f5f5f5; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #667eea; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <strong style="color: #5e35b1; font-size: 14px;">🤖 Agent:</strong>
                    <div style="margin: 8px 0 0 0; color: #212121; font-size: 14px; line-height: 1.8;">{formatted_response}</div>
                </div>
                """))
            
            # Store in history
            self.chat_history.append({"user": message, "agent": response_text})
            
        except Exception as e:
            # Clear thinking indicator
            self.chat_output.clear_output(wait=True)
            
            # Re-display chat history
            for item in self. chat_history:
                with self.chat_output:
                    display(HTML(f"""
                    <div style="background: #e3f2fd; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #2196f3;">
                        <strong style="color: #1565c0;">💼 You:</strong>
                        <p style="margin: 8px 0 0 0; color: #212121;">{item['user']}</p>
                    </div>
                    <div style="background: #f5f5f5; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #667eea;">
                        <strong style="color: #5e35b1;">🤖 Agent:</strong>
                        <div style="margin: 8px 0 0 0; color: #212121;">{item['agent']}</div>
                    </div>
                    """))
            
            # Display error
            with self.chat_output:
                display(HTML(f"""
                <div style="background: #ffebee; padding: 15px; border-radius: 10px; margin: 10px 0; border-left: 4px solid #f44336; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <strong style="color: #c62828; font-size: 14px;">❌ Error:</strong>
                    <p style="margin: 8px 0 0 0; color: #212121; font-size: 14px;">{str(e)}</p>
                </div>
                """))
    
    def on_clear(self, b):
        """Clear chat history"""
        self.chat_output.clear_output()
        self.chat_history = []
        with self.chat_output:
            display(HTML("""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
                <h2 style="margin: 0; color: white;">🤖 Bank Customer Churn Analytics Agent</h2>
                <p style="margin: 10px 0; color: white; font-size: 16px;">Powered by Google Gemini AI with Optional Web Search</p>
                <p style="font-size: 14px; opacity: 0.95; color: white;">Ask me anything about customer churn patterns and recommendations!</p>
            </div>
            <div style="background: #e8f5e9; padding: 15px; border-radius: 8px; text-align: center; border: 2px solid #4caf50;">
                <span style="color: #2e7d32; font-size: 16px; font-weight: bold;">✅ Chat cleared!  Ask me anything about customer churn. </span>
            </div>
            """))

# Create visual chatbot
def create_visual_chatbot(agent):
    """Create and display visual chatbot"""
    chatbot = VisualChatbot(agent)
    chatbot.display()
    return chatbot

print("✅ Visual chatbot loaded with web search capability!")

✅ Visual chatbot loaded with improved contrast!


## 8. Launch Interactive Chatbot
Instantiate and display the visual chatbot for interactive exploration of churn analysis insights.

In [68]:
# Create and display the improved chatbot
visual_chatbot = create_visual_chatbot(agent)

/tmp/ipykernel_47/1042012250.py:92: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.input_box.on_submit(self.on_send)


## 9. Advanced Q&A Examples (Uncomment to Explore)
Below are commented examples of advanced queries you can uncomment to explore:
- **Geographic Analysis**: Compare churn patterns across different countries and regions
- **Product & Engagement**: Analyze how product holdings and member activity affect churn
- **Risk & Segments**: Identify highest-risk customer profiles and strategic priorities
- **Financial Impact**: Calculate revenue losses and ROI of retention strategies
- **Recommendations**: Get data-driven action items and best practices for retention

Uncomment any question and run the cell to see the agent's detailed analysis and insights.

In [ ]:
# # Geographic Analysis
# agent.ask("Which countries have the highest churn? ")
# agent.ask("Why is Germany churning more than other countries?")

# # Product & Engagement
# agent.ask("How do product holdings affect churn?")
# agent. ask("What about active vs inactive members?")

# # Risk & Segments
# agent.ask("Who are our highest-risk customers?")
# agent.ask("What customer segments should we prioritize?")

# # Financial Impact
# agent.ask("What's the financial impact of churn?")
# agent.ask("How much revenue are we losing? ")
# agent.ask("If we reduce churn by 15%, what's the savings?")

# # Recommendations
# agent.ask("What should we do to reduce churn?")
# agent.ask("Give me your top 5 action items")
# agent.ask("What's our best retention strategy?")

# # Complex Questions
# agent.ask("Compare Germany vs France churn patterns")
# agent.ask("Why do customers with more products churn more?")
# agent.ask("What's the profile of a typical churned customer?")